# Questions
- How to choose the max features/depth
- My model did much worse than theirs; what did I do wrong? Did I remove too many columns? Did theirs overfit? What should I fix?

# Using loan dataset to learn random forest
the data was taken from https://www.lendingclub.com/info/download-data.action  (year 2015)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import timeit

In [2]:
loan_df = pd.DataFrame()
loan_df = pd.read_csv('LoanStats2015.csv')
loan_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,24700.0,24700.0,24700.0,36 months,11.99%,820.28,C,C1,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,35000.0,35000.0,35000.0,60 months,14.85%,829.90,C,C5,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,3600.0,3600.0,3600.0,36 months,13.99%,123.03,C,C4,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,10400.0,10400.0,10400.0,60 months,22.45%,289.91,F,F1,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,20000.0,20000.0,20000.0,60 months,10.78%,432.66,B,B4,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
drop_list = []
# remove columns that have more than 20 different values or more than half is NaN value
for col in loan_df.columns: 
    if(len(loan_df[col].unique()) > 20 or (loan_df[col].isnull().sum()) >= len(loan_df) / 2):
        drop_list.append(col)
        loan_df.drop(columns=col, inplace=True)

print("{} columns were dropped".format(len(drop_list)))

loan_df.head()

123 columns were dropped


,term,grade,emp_length,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,purpose,inq_last_6mths,...,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,num_tl_120dpd_2m,num_tl_30dpd,pub_rec_bankruptcies,hardship_flag,debt_settlement_flag
0,36 months,C,10+ years,MORTGAGE,Not Verified,Dec-15,Fully Paid,n,small_business,4.0,...,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,N
1,60 months,C,10+ years,MORTGAGE,Source Verified,Dec-15,Current,n,debt_consolidation,0.0,...,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,N
2,36 months,C,10+ years,MORTGAGE,Not Verified,Dec-15,Fully Paid,n,debt_consolidation,1.0,...,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,N
3,60 months,F,3 years,MORTGAGE,Source Verified,Dec-15,Fully Paid,n,major_purchase,3.0,...,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,N
4,60 months,B,10+ years,MORTGAGE,Not Verified,Dec-15,Fully Paid,n,home_improvement,0.0,...,0.0,1.0,Joint App,0.0,0.0,0.0,0.0,0.0,N,N


In [4]:
for col in loan_df:
#     print(loan_df[col].nunique() )  
    print("column {} has these unique values:\n{}".format(col, loan_df[col].unique()))
    if(loan_df[col].isnull().sum() > 0):
        print("there are {} NaN values here\n".format(loan_df[col].isnull().sum()))

column term has these unique values:
[' 36 months' ' 60 months' nan]
there are 4 NaN values here

column grade has these unique values:
['C' 'F' 'B' 'E' 'A' 'D' 'G' nan]
there are 4 NaN values here

column emp_length has these unique values:
['10+ years' '3 years' '4 years' '1 year' '7 years' '6 years' '5 years'
 '8 years' '2 years' '9 years' '< 1 year' nan]
there are 23821 NaN values here

column home_ownership has these unique values:
['MORTGAGE' 'RENT' 'OWN' 'ANY' nan]
there are 4 NaN values here

column verification_status has these unique values:
['Not Verified' 'Source Verified' 'Verified' nan]
there are 4 NaN values here

column issue_d has these unique values:
['Dec-15' 'Nov-15' 'Oct-15' 'Sep-15' 'Aug-15' 'Jul-15' 'Jun-15' 'May-15'
 'Apr-15' 'Mar-15' 'Feb-15' 'Jan-15' nan]
there are 4 NaN values here

column loan_status has these unique values:
['Fully Paid' 'Current' 'Charged Off' 'Late (31-120 days)'
 'Late (16-30 days)' 'In Grace Period' 'Default' nan]
there are 4 NaN values

In [5]:
loan_df.tail()

,term,grade,emp_length,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,purpose,inq_last_6mths,...,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,num_tl_120dpd_2m,num_tl_30dpd,pub_rec_bankruptcies,hardship_flag,debt_settlement_flag
421094,36 months,B,10+ years,RENT,Verified,Jan-15,Fully Paid,n,credit_card,1.0,...,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,N
421095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# removing the last 4 rows where the entire row is NaN
loan_df = loan_df[:421095]
loan_df.tail()

,term,grade,emp_length,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,purpose,inq_last_6mths,...,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,num_tl_120dpd_2m,num_tl_30dpd,pub_rec_bankruptcies,hardship_flag,debt_settlement_flag
421090,36 months,B,8 years,RENT,Verified,Jan-15,Fully Paid,n,debt_consolidation,0.0,...,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,N
421091,36 months,B,10+ years,MORTGAGE,Verified,Jan-15,Fully Paid,n,home_improvement,1.0,...,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,N
421092,60 months,D,5 years,RENT,Verified,Jan-15,Charged Off,n,debt_consolidation,0.0,...,0.0,1.0,Individual,0.0,0.0,0.0,0.0,1.0,N,N
421093,60 months,E,1 year,RENT,Source Verified,Jan-15,Charged Off,n,debt_consolidation,2.0,...,1.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,N
421094,36 months,B,10+ years,RENT,Verified,Jan-15,Fully Paid,n,credit_card,1.0,...,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,N


from above, we saw that two columns with the most NaN values are emp_length and num_tl_120dpd_2m. </br>
Next, check how much in percentage it's; if it's not too much, all the NaN values will be dropped. Else the columns might need to be omitted.

In [7]:
NaN_values = loan_df['emp_length'].isnull().sum() + loan_df['num_tl_120dpd_2m'].isnull().sum()
print("The max number of rows (if not 0 overlap) with NaN values: % d" % (NaN_values))
print("Which is {}% of the total dataset".format((NaN_values / len(loan_df)) * 100))

The max number of rows (if not 0 overlap) with NaN values:  43047
Which is 10.222633847469098% of the total dataset


In [8]:
# I will drop all the NaN row, since we have enough data also without it.
print("Before dropping there are % d rows" % len(loan_df))
loan_df.dropna(inplace=True)
print("After dropping there are % d rows" % len(loan_df))
loan_df.head()

Before dropping there are  421095 rows
After dropping there are  379318 rows


,term,grade,emp_length,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,purpose,inq_last_6mths,...,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,num_tl_120dpd_2m,num_tl_30dpd,pub_rec_bankruptcies,hardship_flag,debt_settlement_flag
0,36 months,C,10+ years,MORTGAGE,Not Verified,Dec-15,Fully Paid,n,small_business,4.0,...,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,N
1,60 months,C,10+ years,MORTGAGE,Source Verified,Dec-15,Current,n,debt_consolidation,0.0,...,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,N
2,36 months,C,10+ years,MORTGAGE,Not Verified,Dec-15,Fully Paid,n,debt_consolidation,1.0,...,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,N
3,60 months,F,3 years,MORTGAGE,Source Verified,Dec-15,Fully Paid,n,major_purchase,3.0,...,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,N
4,60 months,B,10+ years,MORTGAGE,Not Verified,Dec-15,Fully Paid,n,home_improvement,0.0,...,0.0,1.0,Joint App,0.0,0.0,0.0,0.0,0.0,N,N


In [9]:
# separating to dependent and independent
X = loan_df.drop('loan_status', 1)
y = loan_df['loan_status']

# create dummy variables 
X = pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [10]:
# decision tree
decision_tree = tree.DecisionTreeClassifier(
    criterion='entropy',
#    max_features=1,
#    max_depth=4,
)

start = timeit.default_timer()

decision_tree.fit(X_train, y_train)
decision_tree.score(X_test, y_test)

stop = timeit.default_timer()
print('Time for decision tree: ', stop - start)  

#cross_val_score(decision_tree, X, y, cv=10)

Time for decision tree:  4.6506343


In [11]:
rfc = ensemble.RandomForestClassifier()

start = timeit.default_timer()

rfc.fit(X_train, y_train)
rfc.score(X_test, y_test)

stop = timeit.default_timer()
print('Time for random forest: ', stop - start)  

#cross_val_score(rfc, X, y, cv=10)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Time for random forest:  7.2698145000000025
